In [145]:
import os
import polars as pl

In [146]:
pl.Config(tbl_rows=200)

In [309]:
df = pl.read_parquet(
    os.path.join("data","fdb.parquet")
).with_columns(
    pl.col("datum").dt.month().alias("mesic")
).with_columns(
    pl.col("datum").dt.week().alias("tyden")
).with_columns(
    pl.col("datum").dt.year().alias("rok")
).with_columns(
    pl.when(
        pl.col("url").str.contains("serial/")
    ).then(
        pl.lit("serial")
    ).when(
        pl.col("url").str.contains("film/")
    ).then(
        pl.lit("film")
    ).alias(
        "kategorie"
    )
).sort(
    by="datum"
)

In [148]:
df.sample(10)

kanal,datum,nazev,extra,noticka,url,mesic,tyden,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,str
"""CS Film""",2008-08-20 12:00:00,"""Cesta ke štěstí (?)""","""06.06.2024 v 16:00 (CS Film)""","""Další díl/opakování:06.06.2024…","""film/cesta-ke-stesti/5053""",8,34,"""film"""
"""Film+""",2012-12-26 06:10:00,"""Super náhradník""","""08.11.2024 v 20:20 (Film+)10.1…","""Další díl/opakování:08.11.2024…","""film/super-nahradnik-she-s-the…",12,52,"""film"""
"""Nova Cinema""",2011-04-30 01:30:00,"""Prokletí""","""08.09.2024 v 22:00 (Film+)12.0…","""Další díl/opakování:08.09.2024…","""film/prokleti-cursed/39769""",4,17,"""film"""
"""Prima Cool""",2013-04-26 22:15:00,"""Hrozba z temnot""",null,"""Horor USA/Kan./Fr. (2009). Pař…","""film/hrozba-z-temnot-rise-of-t…",4,17,"""film"""
"""ČT2""",2007-07-04 21:10:00,"""Krásná Praha olympijská""",null,null,null,7,27,null
"""ČT 3""",2021-06-03 01:25:00,"""Náš venkov""","""14.04.2025 v 03:45 (ČT2)15.04.…","""Další díl/opakování:14.04.2025…","""film/nas-venkov/73749""",6,22,"""film"""
"""Nova Action""",2019-09-08 16:05:00,"""Mistři zastavárny XVI""","""15.04.2025 v 14:20 (Nova Actio…","""Další díl/opakování:15.04.2025…","""film/hvezdy-zastavarny-pawn-st…",9,36,"""film"""
"""Nova Fun""",2025-02-03 10:40:00,"""Teleshopping""",null,"""Každý rok se musí dostat k dět…",null,2,6,null
"""NOVA""",2015-01-21 17:00:00,"""Odpolední Televizní noviny, Sp…",null,null,null,1,4,null


In [149]:
kanaly = df.select(pl.col("kanal")).unique().to_series().to_list()
kanaly

['Cinemax',
 'ČT2',
 'Nova Lady',
 'Film Europe Channel',
 'Nova Action',
 'Prima Show',
 'Prima',
 'CS Film',
 'Prima LOVE',
 'NOVA',
 'ČT1',
 'Nova 2',
 'Film+',
 'Cinemax2',
 'Prima STAR',
 'Prima Cool',
 'TV Barrandov',
 'Nova Cinema',
 'Nova Fun',
 'VOYO',
 'CNN Prima News',
 'ČT 3']

In [150]:
verejnopravni = [k for k in kanaly if 'ČT' in k]
print(verejnopravni)
soukrome = [k for k in kanaly if 'ČT' not in k]
print(soukrome)

['ČT2', 'ČT1', 'ČT 3']
['Cinemax', 'Nova Lady', 'Film Europe Channel', 'Nova Action', 'Prima Show', 'Prima', 'CS Film', 'Prima LOVE', 'NOVA', 'Nova 2', 'Film+', 'Cinemax2', 'Prima STAR', 'Prima Cool', 'TV Barrandov', 'Nova Cinema', 'Nova Fun', 'VOYO', 'CNN Prima News']


In [151]:
vysilane_verejnopravni = df.filter(pl.col("kanal").is_in(verejnopravni)).select(pl.col("nazev")).unique().to_series().to_list()
vysilane_soukromymi = df.filter(pl.col("kanal").is_in(soukrome)).select(pl.col("nazev")).unique().to_series().to_list()
vysilane_vsude = [x for x in vysilane_verejnopravni if x in vysilane_soukromymi]

In [152]:
x_let = df.with_columns(pl.col('datum').dt.year().alias('rok')).group_by('nazev').agg(pl.col('rok').unique().len()).filter(pl.col('rok') >= 5).select(pl.col('nazev')).unique().to_series().to_list()

In [153]:
vsude_a_x_let = [x for x in x_let if x in vysilane_vsude]

In [253]:
zajima_nas = ['ČT1', 'ČT2', 'NOVA', 'Prima']

In [255]:
letni = df.filter(pl.col('kanal').is_in(zajima_nas)).filter(pl.col('nazev').is_in(vsude_a_x_let) & pl.col('mesic').is_in([7,8])).group_by('nazev').len().rename({'len':'v_lete'})

In [257]:
zimni = df.filter(pl.col('kanal').is_in(zajima_nas)).filter(pl.col('nazev').is_in(vsude_a_x_let) & pl.col('mesic').is_in([1,2,3,10,11,12])).group_by('nazev').len().rename({'len':'v_zime'})

In [259]:
kategorie = df.filter(pl.col("nazev").is_in(vsude_a_x_let)).group_by("nazev").agg(pl.col("kategorie").mode()).explode("kategorie").drop_nulls()
kategorie = df.filter(pl.col("nazev").is_in(vsude_a_x_let)).group_by(['nazev','kategorie']).len().sort(by='len',descending=True).unique(subset=['nazev'], keep='first').drop('len').drop_nulls()
kategorie

nazev,kategorie
str,str
"""Sedm""","""film"""
"""Superpolda""","""film"""
"""Přijela k nám pouť""","""film"""
"""Na samotě u lesa""","""film"""
"""Záhada bermudského trojúhelník…","""film"""
"""Sanitka 2""","""serial"""
"""Přísaha""","""film"""
"""Beznaděj""","""film"""
"""Jsi falešný hráč""","""film"""


In [261]:
pomer = letni.join(zimni, how='full', on='nazev').with_columns(
    pl.when(
        pl.col('nazev').is_null()
    ).then(
        pl.col('nazev_right')
    ).otherwise(
        pl.col('nazev')
    ).alias(
        'nazev'
    )
).with_columns(
    pl.col('v_lete').fill_null(0)
).drop(
    'nazev_right'
).with_columns(
    (pl.col('v_lete') / pl.col('v_zime') * 3).alias('pomer')
)

In [283]:
filmy = kategorie.filter(pl.col('kategorie') == 'film').select(pl.col('nazev')).to_series().to_list()
serialy = kategorie.filter(pl.col('kategorie') == 'serial').select(pl.col('nazev')).to_series().to_list()

In [273]:
pomer.filter(pl.col('nazev').str.contains('Hříšný t'))

nazev,v_lete,v_zime,pomer
str,u32,u32,f64
"""Hříšný tanec 2""",0,3,0.0
"""Hříšný tanec""",5,10,1.5


In [277]:
pomer.filter(pl.col('nazev').is_in(filmy)).filter(pl.col('v_lete') >= 8).sort(by='pomer',descending=True).head(20)

nazev,v_lete,v_zime,pomer
str,u32,u32,f64
"""Pearl Harbor""",9,null,null
"""Pinocchiova dobrodružství""",26,null,null
"""Ztráta paměti""",13,null,null
"""Rebelové""",14,1,42.0
"""Eden""",10,1,30.0
"""Vinnetou""",18,2,27.0
"""Poklad na Stříbrném jezeře""",18,2,27.0
"""Šaráda""",20,3,20.0
"""Vinnetou - Rudý gentleman""",12,2,18.0


In [289]:
pomer.filter(pl.col('nazev').is_in(serialy)).filter(pl.col('v_lete') >= 50).sort(by='pomer',descending=True).head(20)

nazev,v_lete,v_zime,pomer
str,u32,u32,f64
"""Máma""",85,null,null
"""Malý pitaval z velkého města""",188,15,37.6
"""Chalupáři""",191,21,27.285714
"""Pan Tau""",78,13,18.0
"""Dobrá Voda""",62,13,14.307692
"""Vinaři""",151,52,8.711538
"""Helena""",129,54,7.166667
"""Tělo jako důkaz""",90,60,4.5
"""Sanitka""",73,61,3.590164


In [291]:
pomer.filter(~pl.col('nazev').is_in(serialy) & ~pl.col('nazev').is_in(filmy)).filter(pl.col('v_lete') >= 50).sort(by='pomer',descending=True).head(20)

nazev,v_lete,v_zime,pomer
str,u32,u32,f64
"""Hodinový manžel""",72,66,3.272727
"""Svět zvířat""",78,93,2.516129
"""Z očí do očí""",96,142,2.028169
"""Ochránce""",181,385,1.41039
"""Zprávy""",633,1520,1.249342
"""Televizní noviny, Sportovní no…",311,844,1.10545
"""Počasí""",372,1265,0.882213
"""Klíč""",67,495,0.406061


In [159]:
df.filter(pl.col("nazev") == "Kojak")

kanal,datum,nazev,extra,noticka,url,mesic,tyden,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,str
"""ČT 3""",2022-10-18 21:25:00,"""Kojak""","""21.10.2022 v 22:30 (ČT 3)22.10…","""Další díl/opakování:21.10.2022…","""serial/kojak/44661""",10,42,"""serial"""
"""NOVA""",2007-05-17 11:35:00,"""Kojak""","""II série, díl 3""",null,null,5,20,null
"""ČT1""",2016-02-19 10:45:00,"""Kojak""","""21.10.2022 v 22:30 (ČT 3)22.10…","""Další díl/opakování:21.10.2022…","""serial/kojak/44661""",2,7,"""serial"""
"""ČT 3""",2022-09-06 17:10:00,"""Kojak""","""21.10.2022 v 22:30 (ČT 3)22.10…","""Další díl/opakování:21.10.2022…","""serial/kojak/44661""",9,36,"""serial"""
"""ČT1""",2016-08-03 15:25:00,"""Kojak""","""21.10.2022 v 22:30 (ČT 3)22.10…","""Další díl/opakování:21.10.2022…","""serial/kojak/44661""",8,31,"""serial"""
"""NOVA""",2007-05-14 11:35:00,"""Kojak""","""díl 22""",null,null,5,20,null
"""ČT 3""",2022-09-06 22:35:00,"""Kojak""","""21.10.2022 v 22:30 (ČT 3)22.10…","""Další díl/opakování:21.10.2022…","""serial/kojak/44661""",9,36,"""serial"""
"""ČT 3""",2022-08-20 18:00:00,"""Kojak""","""21.10.2022 v 22:30 (ČT 3)22.10…","""Další díl/opakování:21.10.2022…","""serial/kojak/44661""",8,33,"""serial"""
"""ČT 3""",2022-07-28 23:05:00,"""Kojak""","""21.10.2022 v 22:30 (ČT 3)22.10…","""Další díl/opakování:21.10.2022…","""serial/kojak/44661""",7,30,"""serial"""


In [225]:
kategorie

nazev,kategorie
str,str
"""Pán prstenů""","""film"""
"""Televizní noviny 14. 10. 2020""","""film"""
"""Kazatel""","""film"""
"""Útěk ze seriálu (?) [TV hra]""","""film"""
"""Štvanci""","""film"""
"""To je vražda, napsala: Konečně…","""serial"""
"""Poutníci (?) [TV inscenace]""","""film"""
"""Všichni jsou v pohodě""","""film"""
"""Betlém - naděje lužní krajiny""","""film"""


In [245]:
vyber_strojovy = pomer.filter(pl.col('v_lete') >= 15).sort(by='pomer',descending=True).head(50).select(pl.col('nazev')).to_series().to_list()
graf = df.filter(pl.col('nazev').is_in(vyber_strojovy)).group_by(['nazev','tyden']).len().join(
    kategorie, on="nazev", how="left"
)

graf.sort(by=['nazev','tyden'])

nazev,tyden,len,kategorie
str,i8,u32,str
"""Bouřlivé víno""",1,1,"""film"""
"""Bouřlivé víno""",3,1,"""film"""
"""Bouřlivé víno""",7,1,"""film"""
"""Bouřlivé víno""",9,2,"""film"""
"""Bouřlivé víno""",12,1,"""film"""
"""Bouřlivé víno""",14,1,"""film"""
"""Bouřlivé víno""",15,1,"""film"""
"""Bouřlivé víno""",18,2,"""film"""
"""Bouřlivé víno""",19,2,"""film"""


nazev,tyden,len
str,i8,u32
"""Policajt""",31,2
"""Pan Tau""",2,2
"""Stopaři""",6,1
"""Protiúder""",17,8
"""Šifra Karla Velikého""",17,1
"""Vinaři""",35,7
"""Bouřlivé víno""",41,2
"""Helena""",1,1
"""Vinnetou a míšenka Apanači""",36,1


In [162]:
import altair as alt

In [163]:
df.sample(10)

kanal,datum,nazev,extra,noticka,url,mesic,tyden,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,str
"""Film+""",2018-09-06 05:20:00,"""Zoom In""","""21.01.2025 v 05:40 (Film+)23.0…","""Další díl/opakování:21.01.2025…","""serial/zoom-in/153252""",9,36,"""serial"""
"""Nova 2""",2021-07-23 12:35:00,"""Přátelé""","""18.01.2025 v 21:40 (Nova Fun)1…","""Další díl/opakování:18.01.2025…","""serial/pratele-friends/29320""",7,29,"""serial"""
"""Nova Cinema""",2021-01-31 23:30:00,"""Nezvaní hosté""","""01.07.2021 v 22:30 (Nova Cinem…","""Další díl/opakování:01.07.2021…","""film/nezvani-hoste-intruders/8…",1,4,"""film"""
"""Cinemax2""",2015-04-14 15:55:00,"""Tisíc dnů s Annou""","""24.06.2023 v 20:00 (ČT2)""","""Další díl/opakování:24.06.2023…","""film/tisic-dnu-s-annou-anne-of…",4,16,"""film"""
"""Prima Cool""",2024-08-05 04:55:00,"""Top Gear""","""21.01.2025 v 03:10 (Prima Cool…","""Další díl/opakování:21.01.2025…","""film/top-gear/50964""",8,32,"""film"""
"""ČT2""",2010-07-24 13:15:00,"""Sabotáž""",null,"""Víte, co chcete? Dělejte to! V…",null,7,29,null
"""ČT1""",2021-10-11 12:00:00,"""Zprávy ve 12""",null,null,null,10,41,null
"""Prima Cool""",2014-07-24 21:20:00,"""Teorie velkého třesku""","""18.01.2025 v 20:00 (Nova Fun)1…","""Další díl/opakování:18.01.2025…","""serial/teorie-velkeho-tresku-t…",7,30,"""serial"""
"""ČT2""",2021-01-25 02:30:00,"""Revival""",null,"""Dokument (2010). Bezejmenní um…",null,1,4,null


In [251]:
df.filter(pl.col('nazev').str.contains('Praštěná'))

kanal,datum,nazev,extra,noticka,url,mesic,tyden,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,str
"""ČT1""",2008-07-30 14:40:00,"""Praštěná holka""","""díl 14""",null,null,7,31,null
"""ČT1""",2008-08-25 14:40:00,"""Praštěná holka""","""II série, díl 10""",null,null,8,35,null
"""ČT1""",2009-09-09 13:50:00,"""Praštěná holka II""","""8/22""",null,null,9,37,null
"""ČT1""",2009-09-11 13:30:00,"""Praštěná holka II""","""11/22""",null,null,9,37,null
"""ČT1""",2009-09-10 13:30:00,"""Praštěná holka II""","""9/22""",null,null,9,37,null
"""ČT1""",2008-09-02 14:35:00,"""Praštěná holka""","""II série, díl 15""",null,null,9,36,null
"""NOVA""",2023-05-20 13:15:00,"""Praštěná holka""","""02.03.2025 v 13:15 (Paramount …","""Další díl/opakování:02.03.2025…","""film/prastena-holka-cluelless/…",5,20,"""film"""
"""ČT1""",2009-09-01 13:30:00,"""Praštěná holka""","""I série, díl 13, 14""",null,null,9,36,null
"""ČT1""",2009-09-08 13:30:00,"""Praštěná holka II""","""5/22""",null,null,9,37,null


In [249]:
kategorie.filter(pl.col('nazev').str.contains('Praštěná'))

nazev,kategorie
str,str


In [247]:
alt.Chart(
    graf
).mark_circle(
).encode(
    alt.X('tyden:Q'),
    alt.Y('nazev:N', sort=vyber_strojovy),
    alt.Size('len:Q'),
    alt.Color('kategorie:N')
).resolve_axis(
    y='independent'
)

alt.Chart(...)

## (dis)kontinuita

In [323]:
df.filter(pl.col('nazev').str.contains('Krotitel')).sample(10)

kanal,datum,nazev,extra,noticka,url,mesic,tyden,rok,kategorie
cat,datetime[μs],str,str,str,str,i8,i8,i32,str
"""ČT1""",2012-03-14 21:45:00,"""Krotitelé dluhů""",null,null,"""film/krotitele-dluhu/84924""",3,11,2012,"""film"""
"""Cinemax2""",2018-02-09 17:00:00,"""Krotitelé duchů""","""26.01.2025 v 07:45 (HBO3)""","""Další díl/opakování:26.01.2025…","""film/krotitele-duchu-ii-ghostb…",2,6,2018,"""film"""
"""ČT1""",2012-01-27 01:15:00,"""Krotitelé dluhů""",null,"""Příběh maminky dvou malých dět…",null,1,4,2012,null
"""Cinemax""",2020-03-17 03:45:00,"""Krotitel Morant""","""20.07.2020 v 03:30 (Cinemax)""","""Další díl/opakování:20.07.2020…","""film/krotitel-morant-breaker-m…",3,12,2020,"""film"""
"""ČT1""",2021-11-22 22:00:00,"""Krotitelé dluhů""",null,"""Pavel Šprincl. Opakování 28.11…","""film/krotitele-dluhu/84924""",11,47,2021,"""film"""
"""ČT1""",2009-11-24 21:40:00,"""Krotitelé dluhů""","""24""",null,null,11,48,2009,null
"""ČT1""",2010-11-04 02:20:00,"""Krotitelé dluhů""","""III (8/16)""","""Chtěli dokázat, že jsou dospěl…",null,11,44,2010,null
"""Prima Cool""",2010-09-02 01:20:00,"""Krotitelé vln""","""Dokument USA/Fr. 2004""",null,null,9,35,2010,null
"""ČT1""",2021-11-29 22:10:00,"""Krotitelé dluhů""",null,"""Zuzana a Ludvík Moudří. Opakov…",null,11,48,2021,null


In [317]:
df.select(pl.col('nazev')).sample(20)

nazev
str
"""M.A.S.H"""
"""Jak to dopadlo!?"""
"""Tyhle šaty beru!"""
"""Columbo"""
"""Poslední chlap na Zemi"""
"""Pátek třináctého"""
"""Král tančí"""
"""Mistři zastavárny XII"""
"""Dobrodružství vědy a techniky"""


In [327]:
def calculate_ratio(current_row: pl.Series, previous_row: pl.Series) -> float:
    current_items = set(current_row)
    previous_items = set(previous_row)

    if not previous_items:
        return 0.0

    common_items = current_items.intersection(previous_items)
    ratio = len(common_items) / len(previous_items)
    return ratio


In [339]:
vyvoj_ct1 = df.filter(pl.col('kanal') == 'ČT1').group_by(['rok','tyden']).agg(pl.col("nazev")).sort(by=['rok','tyden'])

In [341]:
ratios = []
for i in range(1, len(vyvoj_ct1)):
    current_row = vyvoj_ct1["nazev"][i]
    previous_row = vyvoj_ct1["nazev"][i - 1]
    ratio = calculate_ratio(current_row, previous_row)
    ratios.append(ratio)

In [343]:
ratios.insert(0, float('nan'))

In [345]:
vyvoj_ct1 = vyvoj_ct1.with_columns(pl.Series("ratio", ratios))

In [351]:
vyvoj_ct1.filter(pl.col('rok').is_between(2008,2024)).group_by('tyden').agg(pl.col('ratio').median())

tyden,ratio
i8,f64
9,0.705263
25,0.663717
29,0.65
12,0.693069
43,0.71
35,0.610169
7,0.702128
30,0.679144
27,0.587302


In [353]:
alt.Chart(
    vyvoj_ct1.filter(pl.col('rok').is_between(2008,2024)).group_by('tyden').agg(pl.col('ratio').median())
).mark_bar(
).encode(
    alt.X('tyden:Q'),
    alt.Y('ratio:Q')
)

alt.Chart(...)

In [ ]:
def calculate_ratio(current_row: pl.Series, previous_row: pl.Series) -> float:
    current_items = set(current_row)
    previous_items = set(previous_row)

    if not previous_items:
        return 0.0

    common_items = current_items.intersection(previous_items)
    ratio = len(common_items) / len(previous_items)
    return ratio
